In [1]:
import json
import numpy as np

In [2]:
import progressbar
from bert_serving.client import BertClient
bc = BertClient(ip='10.15.82.136')

In [3]:
root_folder = '/home/lyt/code/vqa-concept'
data_folder = '{}/dataTVQA'.format(root_folder)
fea_folder = '{}/image-feature/bottomup'.format(data_folder)
emb_size = 768
ocr_num = 50

In [5]:
#=======================================

In [6]:
split_name = 'test'
jsdata = json.load(open('{}/TextVQA_0.5_{}.json'.format(data_folder, split_name),'r'))
print(jsdata.keys())

dict_keys(['dataset_type', 'dataset_name', 'dataset_version', 'data'])


In [7]:
ocr_asse_file = '{}/ocr_assemble_{}.npy'.format(data_folder, split_name)
ocr_assemble = np.load(ocr_asse_file)

In [8]:
print(ocr_assemble.shape)

(5734,)


In [9]:
data = jsdata['data']
print(len(data),type(data))

5734 <class 'list'>


In [10]:
for i, pair in enumerate(data):
    pair['ocr_tokens'] = ocr_assemble[i]

['NOKIA', 'E71', 'nOperator', '3006.2000', '12)', 'Lnร', 'Busanes', 'report', 'મ.mal()', 'eeling', 'network', 'fวอed', '*4ে', '()', '12) Busanes', 'Lnร Busanes', 'Busanes report', 'network fวอed']
['NOKIA', 'E71', 'nOperator', '3006.2000', '12)', 'Lnร', 'Busanes', 'report', 'મ.mal()', 'eeling', 'network', 'fวอed', '*4ে', '()', '12) Busanes', 'Lnร Busanes', 'Busanes report', 'network fวอed']


In [11]:
json.dump(jsdata, open('{}/TextVQA_0.5_{}.assemble.json'.format(data_folder, split_name),'w'))

In [12]:
#------save im_id:ocr_information------

In [13]:
im_ocr = {}
for pair in data:
    im_id = pair['image_id']
    if im_id not in im_ocr:
        im_ocr[im_id] = {'image_classes': pair['image_classes'],
                         'image_width': pair['image_width'],
                         'image_height': pair['image_height'],
                         'ocr_tokens': pair['ocr_tokens'],
                         'ocr_info': pair['ocr_info']}
#json.dump(im_ocr, open('{}/ocr_{}.json'.format(data_folder, split_name),'w'))

In [14]:
print(len(im_ocr))

3289


In [15]:
im_ids = np.load('{}/{}_100_id.npy'.format(fea_folder, split_name))
print(im_ids.shape)

(3289,)


In [16]:
#ocr_emb = np.random.randn(im_ids.shape[0], ocr_num, emb_size).astype('float32')
#ocr_emb = (ocr_emb + mean) * std
ocr_emb = np.zeros((im_ids.shape[0], ocr_num, emb_size), dtype='float32')
bar = progressbar.ProgressBar(max_value=im_ids.shape[0]).start()
for i,imid in enumerate(im_ids):
    bar.update(i+1)
    all_ocr = im_ocr[imid]['ocr_tokens']
    all_ocr = [ocr.strip().lower() for ocr in all_ocr]
    all_ocr = [ocr for ocr in all_ocr if ocr != '']
    num = min(ocr_num,len(all_ocr))
    if num == 0:
        continue
    ocr_list = [all_ocr[j] for j in range(num)]
    curr_embs = bc.encode(ocr_list)
    ocr_emb[i][:num] = curr_embs
bar.finish()

100% (3289 of 3289) |#####################| Elapsed Time: 0:00:55 Time: 0:00:55


In [17]:
ocr_emb.shape

(3289, 50, 768)

In [18]:
np.save('{}/ocr_bert.new.{}/{}_{}_ocr.assemble.0init.npy'.format(fea_folder, emb_size, split_name, ocr_num), ocr_emb)

In [19]:
#=======================================

In [20]:
bc.close()